In [1]:
import polars as pl
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

/Users/rolando/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pl.read_parquet('../data/juul_nc_emails.parquet')

In [3]:
chunk_size = 10000
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
embeddings = [model.encode(chunk.select("ocr_text").to_series().to_list()) for chunk in chunks]

In [ ]:
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [ ]:
query = "Which celebrities were mentioned in the emails?"
query_embedding = model.encode(query)
k = 5  # Number of results to retrieve
distances, indices = index.search(np.array([query_embedding]), k)